In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
# some basic imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns

In [ ]:

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Jordan/Data"

In [ ]:
#changing the directory to the data directory
%cd /content/drive/My\ Drive/Jordan/Data

In [ ]:
class DataShell:
  def __init__(self):
    """Initializes the class with some information about the data and the model"""
    self.data_used = 'The data used is the transformed insurance data'
    

  def read_data(self,filename):
    """Reads the data from the current directory
      parameters: filename in the directory
      returns: Dataframe of the data
    """
    self.data = pd.read_stata(filename)
    return self.data

  def read_data_csv(self,filename):
    self.data2 = pd.read_csv(filename)
    return self.data2

  def get_initial_data(self):
    """Returns the initial dataframe in its current form"""
    return self.data

  def var_init(self):
    self.data['Switch_down'] = 0
    self.data['Switch_up'] = 0
    self.data['Addition'] = 0
    self.data['VIP_Plan'] = 0
    self.data['Economy_Plan'] = 0
    self.data['Stay_high'] = 0
    self.data['Stay_low'] = 0
    
  def replace_data(self):
    self.data1 = self.data

        
  def clean_data(self):
    self.data1['endo_type'].replace('', np.nan, inplace=True)
    self.data1['network_type'].replace('', np.nan, inplace=True)
    self.data1.dropna(subset=['endo_type','network_type'], inplace=True)
    self.data1.reset_index(drop=True, inplace=True)
        
  def network_coding(self):
    vip = ['General Network','Comprehensive Network']
    eco = ['Super-Restricted Network','Restricted Network','Workers Network']
    for i in range(len(self.data1)):
        if self.data1.loc[i,'network_type'] in vip:
            self.data1.loc[i,'VIP_Plan'] = 1       
        else:
            self.data1.loc[i,'Economy_Plan'] = 1
    return self.data1

  def divide_data(self):
    self.data2.sort_values('principal_hof',inplace=True)
    self.data2.reset_index(drop=True, inplace=True)
    self.data3 = self.data2[:315594]
    self.data4 = self.data2[315594:]

  def DelUnnamedCol(self):
    self.data2 = self.data2.drop('Unnamed: 0',1)


  def switch_coding1(self,data):
    i = 0
    k1 = pd.DataFrame(columns=data.columns)
    uni_prin_hof = data['principal_hof'].unique()
    val_counts = data['principal_hof'].value_counts().to_dict()
    for j,p_hof in enumerate(uni_prin_hof):
      print(j/len(uni_prin_hof)*100)
      k2 = data[i:i+val_counts[p_hof]]
      if len(k2['endo_type'].unique()) == 1:
        if k2['VIP_Plan'].tolist().count(1) == len(k2):
          k2['Stay_high'] = 1
        if k2['Economy_Plan'].tolist().count(1) == len(k2):
          k2['Stay_low'] = 1
        if 'Initial New' in k2['endo_type'].unique():
          k1 = k1.append(k2)
          i = i + val_counts[p_hof]
          continue
      if ('Voluntary Deletion' or 'Replacement of Lost Card') in k2['endo_type'].unique():
        k2['Switch_down'] = 1
      if ('Change In Category' or 'Error Correction' or 'Elective Cancellation') in k2['endo_type'].unique():
        k2['Switch_up'] = 1
      if 'Addition' in k2['endo_type'].unique():
        k2['Addition'] = 1
      k1 = k1.append(k2)
      i = i + val_counts[p_hof] 
    return k1


In [ ]:
#Creates an object for the class and reads the data from the directory
data_shell = DataShell()
data = data_shell.read_data('merged_all1415.dta')
#data.head()
print("The data has ",data.shape[0]," rows and ",data.shape[1]," atrributes")

In [ ]:
data.head()

,payer,master_contract,contract,policynumber,effictive_date,exp_date,endorsment_id,endo_type,endo_date,issue_date,principal_hof,beneficiaryid,beneficiary,card_num,nationality_no,network_type,network,policy_status,empno,dependency,gender,nationality,marital_status,dob,product,category,net_premium,gross_premium,age,currency,mastercontract,branch,reinsurer,effictivedate,expirydate,policyno,maritalstatus,ageband,benef_status,networktype,reportingauthority,txndate,dischargedate,los,visittype_fob,memberstartdate,chronic,preexisting,claimtype,receptiondate,beneficiary_id,authorizationnbr,claimid,invoicenbr,resubmission,providercountry,provideremirate,providertype,providergroup,provider,claimedamount,beneficiartshe_coinsurance,totalcimaiedam,totalpayable,payershare,ben_share_approved,deductionamount,recovery_gartia,recoverytype,payable_non_ip_con,po,orderdate,cheque_nbr,settledate,status,claimcatogory,icdchapter,icd_subchapter,assessment,recoverynote
0,Al Hilal Takaful,Global Force Capital Group,Global Force Capital Group,100-2-NS-X-1071,8/24/2015,8/24/2016,1.0,Initial New,24-Aug-15,31-Aug-15,Hatem Wahid Fawzy,71718.0,Hatem Wahid Fawzy,1703-F85A-6513-D802,784-1961-6595147-4,General Network,Hilal Tak.-GN-GEN. NET,Active,,Principal,Male,Egypt,Married,31-Dec-61,Global Force Male/UMF - Cat A,Cat * A,5683.950195,6687.0,54.0,Arab Emirates Dirham,Global Force Capital Group,Abu Dhabi (H.O),NaN,8/24/2015,8/24/2016,100-2-NS-X-1071,Married,Age 46-55,Active,General Network,HAAD,11/24/2015,11/24/2015,NaN,Out-Patient,24-Aug-15,No,No,Network,11/30/2015,1703,,21734024,122407,No,United Arab Emirates,Abu Dhabi,Polyclinic/Diagnostic Center,Group Not defined,Exeter Medical Center - Abu Dhabi,291.000000,50.0,341.000000,291.000000,291.000000,50.000000,0.000000,0.0,,1.0,108686.0,NaN,,,PO Issued,To Be Paid,Diseases of the musculoskeletal system and con...,Other joint disorders (M20-M25) (734-736.9) (7...,"719.47 Pain in joint, ankle and foot",NaN
1,Al Hilal Takaful,Global Force Capital Group,Global Force Capital Group,100-2-NS-X-1071,8/24/2015,8/24/2016,1.0,Initial New,24-Aug-15,31-Aug-15,Hatem Wahid Fawzy,71718.0,Hatem Wahid Fawzy,1703-F85A-6513-D802,784-1961-6595147-4,General Network,Hilal Tak.-GN-GEN. NET,Active,,Principal,Male,Egypt,Married,31-Dec-61,Global Force Male/UMF - Cat A,Cat * A,5683.950195,6687.0,54.0,Arab Emirates Dirham,Global Force Capital Group,Abu Dhabi (H.O),NaN,8/24/2015,8/24/2016,100-2-NS-X-1071,Married,Age 46-55,Active,General Network,HAAD,10/21/2015,10/21/2015,NaN,Out-Patient,24-Aug-15,No,No,Network,11/15/2015,1703,NASAUH-1510467,21548936,PAS5081152,No,United Arab Emirates,Abu Dhabi,Pharmacy,NMC Group,NMC Pharmacy - Branch - Abu Dhabi,93.099998,0.0,93.099998,93.099998,93.099998,0.000000,0.000000,0.0,,0.0,106714.0,NaN,197865,2/4/2016,Settled,Paid,Diseases of the genitourinary system (ICD9CM 5...,Renal tubulo-interstitial diseases (N10-N16) (...,593.89 Other specified disorder of kidney and ...,NaN
2,Al Hilal Takaful,Global Force Capital Group,Global Force Capital Group,100-2-NS-X-1071,8/24/2015,8/24/2016,1.0,Initial New,24-Aug-15,31-Aug-15,Hatem Wahid Fawzy,71718.0,Hatem Wahid Fawzy,1703-F85A-6513-D802,784-1961-6595147-4,General Network,Hilal Tak.-GN-GEN. NET,Active,,Principal,Male,Egypt,Married,31-Dec-61,Global Force Male/UMF - Cat A,Cat * A,5683.950195,6687.0,54.0,Arab Emirates Dirham,Global Force Capital Group,Abu Dhabi (H.O),NaN,8/24/2015,8/24/2016,100-2-NS-X-1071,Married,Age 46-55,Active,General Network,HAAD,10/24/2015,10/24/2015,NaN,Out-Patient,24-Aug-15,No,No,Network,10/26/2015,1703,,21328417,118171,No,United Arab Emirates,Abu Dhabi,Polyclinic/Diagnostic Center,Group Not defined,Exeter Medical Center - Abu Dhabi,291.000000,50.0,341.000000,291.000000,291.000000,50.000000,0.000000,0.0,,1.0,107092.0,NaN,200400,2/22/2016,Settled,Paid,Diseases of the musculoskeletal system and con...,Other joint disorders (M20-M25) (734-736.9) (7...,"719.47 Pain in joint, ankle and foot",NaN
3,Al Hilal Takaful,Global Force Capital Group,Global Force 

In [ ]:
data_shell.var_init()
data_shell.replace_data()
data_shell.clean_data()

In [ ]:
data11 = data_shell.data1

In [ ]:
data11.Addition.unique()

array([0])

In [ ]:
data1 = data_shell.network_coding()

In [ ]:
#data1.to_csv('Plan_Switching_NetworkCoded.csv')

In [ ]:
data_shell = DataShell()
data2 = data_shell.read_data_csv('Plan_Switching_NetworkCoded.csv')
data_shell.DelUnnamedCol()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
data2.Addition.unique()

array([0])

In [ ]:
data_shell.divide_data()
data3 = data_shell.data3
data4 = data_shell.data4

In [ ]:
data1.groupby(["endo_type", "VIP_Plan"]).size().reset_index(name="Time")

,endo_type,VIP_Plan,Time
0,Addition,0,40236
1,Addition,1,37264
2,Change In Category,0,1400
3,Change In Category,1,3469
4,Elective Cancellation,0,409
5,Elective Cancellation,1,106
6,Error Correction,0,844
7,Error Correction,1,1575
8,Initial New,0,222140
9,Initial New,1,285296
